# Day 24: Arithmetic Logic Unit
https://adventofcode.com/2021/day/24

### Part 1

To enable as many submarine features as possible, find the largest valid fourteen-digit model number that contains no 0 digits. What is the largest model number accepted by MONAD?

In [1]:
import math
import operator
import re
from dataclasses import dataclass, replace
from datetime import datetime
from typing import List, Union
from functools import cache
from advent_of_code import utils

%load_ext blackcellmagic

In [2]:
@dataclass(frozen=True, eq=True, order=True)
class Variable:
    name: str
    value: int

    def __post_init__(self):
        if not isinstance(self.value, int):
            raise ValueError(
                "value must contain an int, "
                f"not {type(self.value)} {self.value}"
            )


def inp(name: str, b: Union[Variable, int]) -> Variable:
    """
    Takes variable name and an initial value (int or Variable)
    and returns a Variable with these properties.
    """
    return Variable(name, b.value if isinstance(b, Variable) else b)


def add(a: Variable, b: Union[Variable, int]) -> Variable:
    """
    Takes a variable, a, and another object b, either int or
    another variable, and returns of new variable a with the
    value of a + b
    """
    b_val = b.value if isinstance(b, Variable) else b
    return replace(a, value=a.value + b_val)


def mul(a: Variable, b: Union[Variable, int]) -> Variable:
    """
    Takes variable, a, and another object b, either int or
    another variable, and a becomes a times b.
    """
    b_val = b.value if isinstance(b, Variable) else b
    return replace(a, value=a.value * b_val)


def div(a: Variable, b: Union[Variable, int]) -> Variable:
    """
    Takes variable, a, and another object b, either int or
    another variable, and a becomes a / b rounded down.
    """
    b_val = b.value if isinstance(b, Variable) else b
    if b_val == 0:
        raise ValueError("Cannot divide by 0.")
    return replace(a, value=int(math.floor(a.value / b_val)))


def mod(a: Variable, b: Union[Variable, int]) -> Variable:
    """
    Takes variable, a, and another object b, either int or
    another variable, and a becomes a % b.
    """
    if a.value < 0:
        raise ValueError("Cannot take mod of value < 0")

    b_val = b.value if isinstance(b, Variable) else b
    if b_val <= 0:
        raise ValueError("Cannot take mod of with value <= 0")

    return replace(a, value=a.value % b_val)


def eql(a: Variable, b: Union[Variable, int]) -> Variable:
    """
    Takes variable, a, and another object b, either int or
    another variable, and sets a equal to 1 if a == b, otherwise
    0
    """
    b_val = b.value if isinstance(b, Variable) else b
    return replace(a, value=1 if a.value == b_val else 0)


def value_lookup(
    variable_register: dict, x: Union[str, int]
) -> Union[int, Variable]:
    """
    Takes a string or int and returns the variable with name 'x' or
    int value.
    """

    if x in variable_register.keys():
        # we check if the str or int is a variable name.
        return variable_register.get(x, None)

    elif re.match("[-+]?\d+$", x):
        # we have an integer input
        return int(x)

    else:
        raise ValueError(f"{x} is not a defined variable.")


def exec(variable_register: dict, instruction: str) -> int:
    """Executes the command and returns the value stored in variable."""

    parts: list[str] = instruction.split(" ")
    var1: Variable = value_lookup(variable_register, parts[1])
    var2: Union[Variable, int] = value_lookup(variable_register, parts[2])

    if parts[0] in globals():
        m = globals()[parts[0]]
    else:
        raise NotImplementedError(f"This ALU does not support {parts[0]}.")

    # print(f"{instruction} -> {var1} {var2}")
    variable_register[var1.name] = m(var1, var2)
    return variable_register[var1.name].value


@cache
def check_slice(w: int, z: int, instructions: tuple) -> int:
    """
    execute list of operations (instructions) given w and z starting
    point. this function is cached because we will have collisions and
    we run it looking for largest and smallest.
    """
    variable_register = {
        "w": inp("w", w),
        "x": inp("x", 0),
        "y": inp("y", 0),
        "z": inp("z", z),
    }
    for instruction in instructions:
        exec(variable_register, instruction)
    return variable_register.get("z", None).value


def read_instructions(input_file: str) -> List[str]:
    """Read input file, return a list[str] containing the instructions."""
    ops: list = []
    i: int = -1
    with open(input_file) as f:
        for line in f:
            if line.startswith('inp w'):
                ops.append([])
                i += 1
            elif line.rstrip():
                ops[i].append(line.rstrip())        
    return ops

In [3]:
def look_for_solutions(optimize_largest: bool = True) -> int:
    """
    Using our instruction set (the input), we know that brute force the solution
    by caching all possible solutions. We achieve some speed up because we do a
    breadth first search, keeping only the unique combinations of w and z after
    each instruction set phase (between inp w steps).

    - we decide what to keep based on if we are looking for the largest or
      smallest number.  default is largest and controlled by param
      optimize_largest.

    - w (next sig digit) and z (result from previous instruction set run) are
      important whereas x and y are reset each time.

    - Z numbers that exceed some large number (here 10,000,000) tend to explode
      away from 0, so we cut them from the space to decrease iterations.

    Return a dictionary of model that satisfy. It is not complete, but depending
    on "optimize_largest" we guarentee that largest or smallest will be there.
    """

    instructions = read_instructions(utils.input_location(day=24))

    # key = [sig_dig][(w,z)] = prefix
    cached: list[dict] = []

    for sig_digit, instruction_set in enumerate(instructions):
        print(f"significant digit: {sig_digit} - {datetime.now()}")
        cached.append(dict())

        # determine ending states from previous state.
        previous_zs = {(0, 0): 0} if sig_digit == 0 else cached[sig_digit - 1]

        # iterate over the
        for (input_z, _), prefix in previous_zs.items():
            for w in range(1, 10):
                output_z = check_slice(w, input_z, tuple(instruction_set))
                if output_z > 10000000:
                    continue

                partial_model_number = (prefix * 10) + w
                if not isinstance(partial_model_number, int):
                    print((input_z), (output_z, w), partial_model_number)
                    raise ValueError("")

                key = (output_z, w)

                if (
                    optimize_largest
                    and cached[sig_digit].get(key, 0) < partial_model_number
                ):
                    cached[sig_digit][key] = partial_model_number
                elif not optimize_largest and (
                    cached[sig_digit].get(key, None) is None
                    or cached[sig_digit].get(key, None) > partial_model_number
                ):
                    cached[sig_digit][key] = partial_model_number

    solutions = {
        key: cached[len(cached) - 1][key]
        for key in cached[len(cached) - 1].keys()
        if key[0] == 0
    }
    for key in sorted(solutions, key=operator.itemgetter(1), reverse=optimize_largest):
        return solutions.get(key)

In [4]:
look_for_solutions(optimize_largest=True)
# Part 1 correct answer: 95299897999897

significant digit: 0 - 2022-01-28 17:25:27.564103
significant digit: 1 - 2022-01-28 17:25:27.565060
significant digit: 2 - 2022-01-28 17:25:27.570433
significant digit: 3 - 2022-01-28 17:25:27.619637
significant digit: 4 - 2022-01-28 17:25:28.025316
significant digit: 5 - 2022-01-28 17:25:31.525069
significant digit: 6 - 2022-01-28 17:25:35.433558
significant digit: 7 - 2022-01-28 17:26:09.702701
significant digit: 8 - 2022-01-28 17:26:43.908235
significant digit: 9 - 2022-01-28 17:27:22.763349
significant digit: 10 - 2022-01-28 17:28:06.090084
significant digit: 11 - 2022-01-28 17:28:49.608500
significant digit: 12 - 2022-01-28 17:30:01.735939
significant digit: 13 - 2022-01-28 17:31:20.741672


95299897999897

### Part 2
Find the smallest 14 digit model number that satisfies the result.

In [5]:
# This should run reasonably fast since we cached the results above in Part 1. We only need
# to optimize by keeping the lowest predicate model numbers when we have a collision. 

look_for_solutions(optimize_largest=False)
# Part 2 correct answer: 31111121382151

significant digit: 0 - 2022-01-28 17:32:34.048283
significant digit: 1 - 2022-01-28 17:32:34.048546
significant digit: 2 - 2022-01-28 17:32:34.048647
significant digit: 3 - 2022-01-28 17:32:34.049499
significant digit: 4 - 2022-01-28 17:32:34.058076
significant digit: 5 - 2022-01-28 17:32:34.126079
significant digit: 6 - 2022-01-28 17:32:34.233084
significant digit: 7 - 2022-01-28 17:32:34.911062
significant digit: 8 - 2022-01-28 17:32:35.791561
significant digit: 9 - 2022-01-28 17:32:36.523569
significant digit: 10 - 2022-01-28 17:32:37.705739
significant digit: 11 - 2022-01-28 17:32:38.884509
significant digit: 12 - 2022-01-28 17:32:40.393317
significant digit: 13 - 2022-01-28 17:32:42.773392


31111121382151